In [4]:
import pyblp
import numpy as np
import pandas as pd

pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

# Load the data
sales_data = pd.read_csv("OTC_Sales.csv")
demo_data = pd.read_csv("OTC_Demographics.csv")

In [13]:
#Identify the columns that contain the product sales for each product 
product_cols = sales_data.columns[3:14]

#Now, well start the calculations for the Sum Stats Table. 
# --------------------------------------------------------------#
#Market Shares 
# Convert all product columns to numeric only 
sales_data[product_cols] = sales_data[product_cols].apply(pd.to_numeric, errors='coerce')

# Total sales
total_sales_all_products = sales_data[product_cols].sum().sum()
market_share = {}

# Calculate market share for each product
for product in product_cols:
    total_sales_product = sales_data[product].sum()  # Sum of sales for the product
    market_share[product] = total_sales_product / total_sales_all_products  # Calculate market share

for i, product in enumerate(product_cols, 1):
    print(f"Market Share of Product_{i}: {market_share[product]:.2%}")
# --------------------------------------------------------------#


Market Share of Product_1: 14.49%
Market Share of Product_2: 17.97%
Market Share of Product_3: 11.67%
Market Share of Product_4: 11.88%
Market Share of Product_5: 7.81%
Market Share of Product_6: 3.59%
Market Share of Product_7: 4.06%
Market Share of Product_8: 3.48%
Market Share of Product_9: 8.00%
Market Share of Product_10: 9.37%
Market Share of Product_11: 7.69%


In [19]:
#Unit Price 
price_cols = sales_data.columns[15:26]
unit_price = {}
sales_data[price_cols] = sales_data[price_cols].apply(pd.to_numeric, errors='coerce')

# Calculate unit price for each product
for i, product in enumerate(product_cols, 1):
    # We already have total sales for each product calculated above. 
    # Count total quantity sold for the product
    total_quantity_product = sales_data[product].sum() 
    
    # Get the average price across stores/weeks for each product 
    price_for_product = sales_data[price_cols[i-1]].mean()  
    
    # Calculate unit price (Total Sales / Total Quantity Sold)
    unit_price[f"Product_{i}"] = total_sales_product / total_quantity_product

for product, price in unit_price.items():
    print(f"Unit Price of {product}: ${price:.2f}")


Unit Price of Product_1: $0.53
Unit Price of Product_2: $0.43
Unit Price of Product_3: $0.66
Unit Price of Product_4: $0.65
Unit Price of Product_5: $0.98
Unit Price of Product_6: $2.14
Unit Price of Product_7: $1.89
Unit Price of Product_8: $2.21
Unit Price of Product_9: $0.96
Unit Price of Product_10: $0.82
Unit Price of Product_11: $1.00
